In [ ]:
import requests
import ast
import pandas as pd
import Bio
from Bio import Entrez
from Bio import SeqIO
from time import sleep
from time import monotonic
from orffinder import orffinder
import optipyzer
from Bio.Restriction.Restriction import RestrictionBatch
from IPython.display import clear_output
import re
import json
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [ ]:
def checkPeptides(entry) -> dict:

    transit_present = False
    signal_present = False
    removal_regions = []
    for i in entry['features']:
        if 'Transit' in i['type']:
            transit_present = True
        if 'Signal' in i['type']:
            signal_present = True
        removal_regions.append([i['location']['start']['value'], i['location']['end']['value']])

    return {'transit_peptide': transit_present, 'signal_peptide': signal_present, 'removed_regions': removal_regions}
    

def getUniprotInfo(id) -> dict:

    params = {
        'query': id,
        'fields': ['xref_refseq', 'sequence', 'ft_signal', 'ft_transit']
    }

    print(f'Processing [{id}]')

    res = requests.get(f'https://rest.uniprot.org/uniprotkb/{id}.fasta')
    fasta = res.content.decode('utf-8')
    if fasta == "Error messages\nThe 'accession' value has invalid format. It should be a valid UniProtKB accession":
        print('Issue with uniprot ID. Try removing isoform.')
    uniprotValue = ''.join(fasta.split('\n')[1:])
    les_nam = fasta[re.search('\\|.*\\|', fasta).end():]
    gene_symbol = re.search('(GN=.*? )',fasta)[0][3:].strip()
    name = re.search(' [A-Za-z0-9s][^ ]*', les_nam)[0][1:]

    response = requests.get('https://rest.uniprot.org/uniprotkb/search', params=params)
    try:
        entry = ast.literal_eval(str(response.content.decode('utf-8')))['results'][0]
    except ValueError as e:
        print(e, '\nAttempting to reprocess...')
        entry = ast.literal_eval(str(json.loads(response.content.decode('utf-8'))))['results'][0]
    potential_ids = pd.json_normalize(entry)['uniProtKBCrossReferences'][0]
    sequence_length = entry['sequence']['length']
    if len(entry['features']) > 0:
        peptides = checkPeptides(entry)
        transit_peptide = peptides['transit_peptide']
        signal_peptide = peptides['signal_peptide']
        removed_regions = peptides['removed_regions']
    else:
        transit_peptide, signal_peptide, removed_regions = False, False, pd.NA

    results = {
        'uniprotValue': uniprotValue,
        'potential_ids': potential_ids,
        'sequence_length': sequence_length,
        'gene_symbol': gene_symbol,
        'name': name,
        'transit_peptide': transit_peptide,
        'signal_peptide': signal_peptide,
        'removed_regions': removed_regions
    }
    return results

In [ ]:
def matchAmino(uniprotValue, potential_ids) -> list:
    if potential_ids == None:
        print('No potential ids provided.')
        return 0
    
    pt_ids = [i['properties'][0]['value'] for i in potential_ids]
    potential_ids = [i.split('.')[0] for i in pt_ids] + potential_ids

    for count, test in enumerate(potential_ids):
        print(f'\nChecking {count+1} of {len(potential_ids)} potential matches')
        print(f'Trying to match [{test}]...')

        #ENTER EMAIL HERE
        Entrez.email = ''
        handle = Entrez.efetch(db='nuccore', id=test, rettype='gb', retmode='text')
        sequence = SeqIO.read(handle, "genbank")

        for feature in sequence.features:
            if feature.type=='CDS':
                ncbi_match = feature.qualifiers['translation'][0]

        if ncbi_match == uniprotValue:
            print(f'Match found with [{test}]')
            return [test, sequence, ncbi_match]
        elif count+1 == len(potential_ids):
            continue
        else:
            print('Not a match. Idling for 10 seconds...', end=' ')
            start = monotonic()
            while monotonic()-start < 10:
                print('\u2588', end=' ')
                sleep(1.0 - ((monotonic()-start) % 1.0))

In [ ]:
def matchTranslation(orfs, uniprotValue, sequence) -> str:
    """
    Parameters
    ----------
    orfs : dict
    dict of orfs site from orffinder utility

    uniprotValue : str
    string from uniprot based on id
    """
    
    for i in range(len(orfs)):
        if uniprotValue+'*' == str(sequence.seq[orfs[i]['start']-1:orfs[i]['end']-1].translate()):
            print(f'Matched ORF at {orfs[i]['start']-1} to {orfs[i]['end']}')
            slice = str(sequence.seq[orfs[i]['start']-1:orfs[i]['end']-1])
            return slice

In [ ]:
def optimizeSlice(slice) -> str:
    """
    Parameters
    ----------
    slice : str
    
    get string of appropriate ORF section from matchTranslation
    """

    api = optipyzer.API()
    gblock = str(slice)

    result = api.optimize(
        seq=gblock,
        seq_type="dna",
        weights={"e_coli": 1}
    )

    optimized = result['optimized_sd']
    return optimized


In [ ]:
def findSites(optimizedCodon) -> dict:
    """
    Parameters
    ----------
    optimizedCodon : str
    
    Provider E. Coli optimized codon from optimizedSlice(). Will return dictionary
    with whether enzyme sites are present or not.
    """
    results = {'ins':[], 'outs':[]}
    enzymes = ['BsaI', 'BbsI', 'BsmBI', 'Esp3I', 'SapI']
    batch = RestrictionBatch()
    for i in enzymes:
        batch.add(i)

    for i in enzymes:
        enzyme = batch.get(i)
        if enzyme.site in optimizedCodon:
            results['ins'].append(i)
            print(f'{i} in sequence')
        else:
            results['outs'].append(i)
            print(f'{i} NOT in sequence')

    return results
    

In [ ]:
def removePeptides(sequence, removal_regions) -> str:
    iden = sequence.id
    name = sequence.name
    desc = sequence.description
    sequence = str(sequence.seq)
    sequence = list(sequence)
    to_remove = []
    for i in removal_regions:
        try:
            to_remove = to_remove + [j for j in range(i[0], i[1]*3)]
        except TypeError as e:
            print(e,'\n', f'There was a problem trying to identify peptide start ({i[0]}) or stop ({i[1]})\nRebuilding sequence...')
            sequence = ''.join(sequence)
            record = SeqRecord(
                Bio.Seq.Seq('ATG' + sequence),
                id = iden,
                name = name,
                description = desc
            )
            return record
            
        to_remove = list(set(to_remove))
        to_remove.sort(reverse=True)

    for i in to_remove:
        sequence.pop(i-1)
    
    sequence = ''.join(sequence)
    record = SeqRecord(
        Bio.Seq.Seq('ATG' + sequence),
        id = iden,
        name = name,
        description = desc
    )

    return record
    

In [ ]:
def main(df) -> pd.DataFrame:
    """
    Parameters
    ----------
    df : pd.DataFrame
    csv with uniprot_id column
    """
    
    uniprotId = []
    name = []
    transit_peptide = []
    signal_peptide = []
    removed_regions = []
    symbols = []
    uniprotValue = []
    sequence_length = []
    nih_id = []
    full_sequence = []
    cut_sequence = []
    amino_sequence = []
    orf = []
    optimized_codon = []
    enzymes = []
    

    for count, i in enumerate(df.uniprot_id):
        clear_output(wait=True)
        print(f'Checking {count+1} of {len(df.uniprot_id)}')
        uniprotInfo = getUniprotInfo(i)
        
        try:
            matchedId, sequence, amino = matchAmino(uniprotInfo['uniprotValue'], uniprotInfo['potential_ids'])
        except:
            uniprotValue.append(uniprotInfo['uniprotValue'])
            sequence_length.append(uniprotInfo['sequence_length'])
            nih_id.append(pd.NA)
            full_sequence.append(pd.NA)
            cut_sequence.append(pd.NA)
            amino_sequence.append(pd.NA)
            orf.append(pd.NA)
            optimized_codon.append(pd.NA)
            enzymes.append(pd.NA)
            continue

        if uniprotInfo['transit_peptide'] or uniprotInfo['signal_peptide']:
            cut_seq = removePeptides(sequence, removal_regions=uniprotInfo['removed_regions'])
            orfs = orffinder.getORFs(cut_seq, minimum_length=uniprotInfo['sequence_length'], remove_nested=True)
            cut_seq = cut_seq.seq
            cut_seq = ''.join(cut_seq)
        else:
            cut_seq = pd.NA
            orfs = orffinder.getORFs(sequence, minimum_length=uniprotInfo['sequence_length'], remove_nested=True)
        slice = matchTranslation(orfs, uniprotInfo['uniprotValue'], sequence)
        try:
            codon = optimizeSlice(slice)
            enzyme_results = findSites(optimizedCodon=codon)
        except ValueError as e:
            print('Something\'s wrong with the codon')
            print(e)
            sleep(2)
            codon = e
            enzyme_results = pd.NA
        
        uniprotId.append(i)
        name.append(uniprotInfo['name'])
        transit_peptide.append(uniprotInfo['transit_peptide'])
        signal_peptide.append(uniprotInfo['signal_peptide'])
        removed_regions.append(uniprotInfo['removed_regions'])
        symbols.append(uniprotInfo['gene_symbol'])
        uniprotValue.append(uniprotInfo['uniprotValue'])
        sequence_length.append(uniprotInfo['sequence_length'])
        nih_id.append(matchedId)
        full_sequence.append(str(sequence.seq))
        cut_sequence.append(cut_seq)
        amino_sequence.append(amino)
        orf.append(slice)
        optimized_codon.append(codon)
        enzymes.append(enzyme_results)


        

    result = pd.DataFrame([
        uniprotId,
        name,
        transit_peptide,
        signal_peptide,
        removed_regions,
        symbols,
        uniprotValue,
        sequence_length,
        nih_id,
        full_sequence,
        cut_sequence,
        amino_sequence,
        orf,
        optimized_codon,
        enzymes
    ], index=[
        'uniprotID',
        'name',
        'tr_peptide',
        'si_peptide',
        'peptide_regions',
        'gene',
        'up_sequence',
        'seq_length',
        'nih_id',
        'full_sequence',
        'cut_sequence',
        'amino_sequence',
        'orf',
        'optimized_codon',
        'enzymes'
    ])
    
    result=result.transpose()

    for i in result.index:
        if pd.isna(result.loc[i,'enzymes']):
            continue
        
        cols = result.loc[i, 'enzymes']['ins'] + result.loc[i, 'enzymes']['outs']
        cols.sort()

    for i in cols:
        result[i] = pd.NA

    for i in result.index:
        if pd.isna(result.loc[i, 'enzymes']):
            continue

        for j in result.loc[i, 'enzymes']['ins']:
            result.loc[i, j] = True

        for k in result.loc[i, 'enzymes']['outs']:
            result.loc[i, k] = False

    result = result.drop('enzymes', axis=1)
    
    clear_output(wait=True)
    print('Done')
    return result


In [ ]:
df = pd.read_csv('uniprot_ids.csv')

In [ ]:
result = main(df)

In [ ]:
result.to_csv('results.csv')